In [ ]:
from kaggle_environments import make, evaluate
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
from gym import spaces
import gym
import numpy as np
import random

last_step =[]
def get_nearest_cells(x,y):
    # returns all cells reachable from the current one
    result = []
    for i in (-1,+1):
        result.append(((x+i+7)%7, y))
        result.append((x, (y+i+11)%11))
    return result

def create_table(obs_dict, config_dict):
    global last_step
    
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    
    user_head_val = 1
    user_body_val = -3
    opp_head_move_val = -1
    opp_head_val = -2
    food_val = 2
    forbidden = -5
    delta_val = -4
    funnel_val = -6
    food_pos = []
    opp_head_pos = []
    table = np.zeros((7,11))
    
    for food in observation.food:
        x,y = row_col(food, configuration.columns)
        table[x,y] = food_val
        food_pos.append([x,y])
    # let's add all cells that are forbidden
    for i in range(4):
        opp_goose = observation.geese[i]
        if len(opp_goose) == 0:
            continue
            
        is_close_to_food = False
            
        if i != player_index:
            x,y = row_col(opp_goose[0], configuration.columns)
            table[x,y] = opp_head_val
            opp_head_pos.append([x,y])
            possible_moves = get_nearest_cells(x,y) # head can move anywhere
            
            for x,y in possible_moves:
                if table[x,y] == food_val:
                    is_close_to_food = True
            
                table[x,y] = opp_head_move_val # possibly forbidden cells
        
        # usually we ignore the last tail cell but there are exceptions
        tail_change = -1
        if obs_dict['step'] % 40 == 39:
            tail_change -= 1
        
        # we assume that the goose will eat the food
        if is_close_to_food:
            tail_change += 1
        if tail_change >= 0:
            tail_change = None
            

        for n in opp_goose[1:tail_change]:
            x,y = row_col(n, configuration.columns)
            table[x,y] = -3 # forbidden cells
        for n in player_goose[1:tail_change]:
            x,y = row_col(n, configuration.columns)
            table[x,y] = -3 # forbidden cells
    
    if len(last_step) != 0 :
        table[last_step[0],last_step[1]] = forbidden
    x,y = row_col(player_head, configuration.columns)
    # add head position
    table[x,y] = user_head_val
    last_step = [x,y]
    #delta pos
    for i in range(len(table)):
        for j in range(len(table[i])):
            cells = get_nearest_cells(i,j)
            num,num_1= 0,0
            for k in cells:
                if table[k[0],k[1]] in [-3,-2]:
                    num+=1
                if table[k[0],k[1]] == 1:
                    num_1 += 1
            if num >=3 and table[i,j] not in [-3,-2,-5,1]:
                table[i,j] = delta_val
            if num >=3 and num_1 ==1 and table[i,j] not in [-3,-2,-5]:
                table[i,j] = delta_val
    #funnel_trap pos
    for i in range(len(table)):
        for j in range(len(table[i])):
            cells = get_nearest_cells(i,j)
            num,num_3,num_4 = 0,0,0
            for k in cells:
                if table[k[0],k[1]] in [-3,-4,-2]:
                    num+=1
                if table[k[0],k[1]] == -3:
                    num_3 += 1
                if table[k[0],k[1]] == -4:
                    num_4 += 1
            if num >=3 and num_3 == 2 and num_4 >= 1 and table[i,j] not in [-3,1,-2,-5,-4]:
                table[i,j] = funnel_val   
                funnel_choosen = [i,j] 
        #line_system
                while True:
                    end = True
                    num_3 = 0
                    next_block = get_nearest_cells(funnel_choosen[0],funnel_choosen[1])
                    for n in next_block:
                        if table[n[0],n[1]] == 0:
                            end = False
                            sides = get_nearest_cells(n[0],n[1])
                            for m in sides:
                                if table[m[0],m[1]] in [-3,-2]:
                                    num_3 +=1
                            if num_3 ==2 and table[n[0],n[1]] == 0:
                                table[n[0],n[1]] = funnel_val
                                funnel_choosen = [n[0],n[1]]
                            else:
                                end = True
                    if end:
                        break
    #box_system
    for i in range((7*11)-12):
        if i !=0 and i%6== 0:
            continue
        box_list = [i,i+1,i+11,i+12]
        for j in box_list:
            x,y = row_col(j , configuration.columns)
            cells = get_nearest_cells(x,y)
            num_3,t_num = 0,0
            for k in cells:
                if table[k[0],k[1]] in [-3,-2,1]:
                    num_3 +=1
            if num_3 >=2:
                t_num += 1
        if t_num == 4:
            for j in box_list:
                x,y = row_col(j , configuration.columns)
                table[x,y] = delta_val
    
    return table, food_pos, opp_head_pos, player_goose

def safe_pos(agent,posi_degree_1,posi_degree_2,posi_degree_3):
    if len(posi_degree_1)!= 0:
        step = compute_path_dir(agent, posi_degree_1[0])#into -1
    if len(posi_degree_2)!= 0:
        step = compute_path_dir(agent, posi_degree_2[0])#can go into funnels
    if len(posi_degree_3)!= 0:
        step = compute_path_dir(agent, posi_degree_3[0])#delta
    else:
        step = random.randint(1,4)#last move
    return step
    
def get_dist(p1, p2):
    path0 = abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])
    path1 = 7-abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])
    path2 = abs(p1[0]-p2[0]) + 11-abs(p1[1]-p2[1])
    path3 = 7-abs(p1[0]-p2[0]) + 11-abs(p1[1]-p2[1])
    dis_lis = [path0,path1,path2,path3]
    dist = min(dis_lis)
    return dist
def compute_path_dir(agent, dest):
    #covering X axis first
        if agent[0] != dest[0]:
            if agent[0] > dest[0] and abs(agent[0] - dest[0])==1:
                return 1
            if agent[0] < dest[0] and abs(agent[0] - dest[0])>1:
                return 1
            elif agent[0] < dest[0] and abs(agent[0] - dest[0])==1:
                return 3
            elif agent[0] > dest[0] and abs(agent[0] - dest[0])>1:
                return 3
        else:#Y axis
            if agent[1] > dest[1] and abs(agent[1] - dest[1])==1:
                return 4
            elif agent[1] < dest[1] and abs(agent[1] - dest[1])>1:
                return 4
            elif agent[1] < dest[1] and abs(agent[1] - dest[1])==1:
                return 2
            elif agent[1] > dest[1] and abs(agent[1] - dest[1])>1:
                return 2

def compute_path(agent, dest, posi,lenght,food):
    distance = []
    for i in posi:
        dist = get_dist(i,dest)
        distance.append(dist)
    #if lenght < 10 or food == True:
    final = posi[distance.index(min(distance))]
    #else:
        #final = posi[distance.index(max(distance))]
    return compute_path_dir(agent,final)
            
def food_dist(food_pos,agent,opponents):
    acc,dist = [],100 
    found = False
    for food in food_pos:
        opp_dist_list = []
        user_dist = get_dist(agent,food)
        for j in opponents:
            opp_dist_tem = get_dist(j,food)
            opp_dist_list.append(opp_dist_tem)
        opp_dist = int(min(opp_dist_list))    
        if opp_dist>user_dist and user_dist < dist:
            acc,dist = food,user_dist
            found = True
    if acc == []:
        found = False
    return found, acc
            
def make_move(board, food_pos,opp_head_pos,player_pos):
    player_head = player_pos[0]
    lenght = len(player_pos)
    legend = {
        1: 'NORTH',
        2: 'EAST',
        3: 'SOUTH',
        4: 'WEST'
    }
    step = None
    x,y = row_col(player_head, len(board[0]))
    posi = get_nearest_cells(x,y)
    posi_degree_1 = [i for i in posi if board[i[0],i[1]] not in [-5,-4,-6,-3,-2]]
    posi_degree_2 = [i for i in posi if board[i[0],i[1]] not in [-5,-4,-3,-2]]
    posi_degree_3 = [i for i in posi if board[i[0],i[1]] not in [-5,-3,-2]]
    posi_safe = [i for i in posi if board[i[0],i[1]] in [0,2]]
    center = [3,5]
    found, food = food_dist(food_pos,[x,y],opp_head_pos)
    if len(posi_safe) == 0:
        step = safe_pos([x,y],posi_degree_1,posi_degree_2,posi_degree_3)
    elif found and lenght <= 20:
        step = compute_path([x,y], food,posi_safe,lenght,True)
    elif x == 3 and y == 5:
        step = compute_path_dir([x,y], posi_safe[0])
    elif x != 3 or y != 5:
        step = compute_path([x,y], center, posi_safe,lenght,False)
       
    return legend[step]

def agent(obs_dict, config_dict): #submission agent
    board,food_pos,opp_head_pos,player_pos = create_table(obs_dict,config_dict)
    action = make_move(board,food_pos,opp_head_pos,player_pos)
    return action